In [81]:
pip install geopy

In [210]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.point import Point

In [2]:
movies = pd.read_csv('title.basics.tsv/data.tsv', header=None, sep='\t')
headers = movies.iloc[0]
movies = pd.DataFrame(movies.values[1:], columns=headers)
movies

C:\Users\andyc\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...
7703469,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2010,\N,\N,"Action,Drama,Family"
7703470,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,\N,\N,"Action,Drama,Family"
7703471,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,\N,"Action,Drama,Family"
7703472,tt9916856,short,The Wind,The Wind,0,2015,\N,27,Short


In [3]:
movies = movies[(movies.isAdult == '0') | (movies.isAdult == 0)]
movies = movies[movies.titleType == 'movie']
movies = movies.drop(columns=['titleType', 'originalTitle', 'isAdult', 'endYear'])
movies = movies.rename(columns={"tconst": "movieID", "primaryTitle": "title", "startYear": "year", "runtimeMinutes": "runtime"})

In [4]:
movies

,movieID,title,year,runtime,genres
498,tt0000502,Bohemios,1905,100,\N
570,tt0000574,The Story of the Kelly Gang,1906,70,"Action,Adventure,Biography"
610,tt0000615,Robbery Under Arms,1907,\N,Drama
625,tt0000630,Hamlet,1908,\N,Drama
668,tt0000675,Don Quijote,1908,\N,Drama
...,...,...,...,...,...
7703363,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,57,Documentary
7703390,tt9916680,De la ilusión al desconcierto: cine colombiano...,2007,100,Documentary
7703402,tt9916706,Dankyavar Danka,2013,\N,Comedy
7703413,tt9916730,6 Gunn,2017,116,\N


In [ ]:
genres = movies.set_index('MovieID').genres.str.split(',', expand=True).stack().reset_index('MovieID')
genres = genres.rename(columns={0: "Genre"})
genres = genres[genres.Genre != "\\N"]

In [ ]:
genres

In [ ]:
genres.Genre.value_counts().head(20)

In [74]:
movielocations = pd.read_csv('IMDB-Location.list-Parser/movies.csv', header=None,skiprows = 1,sep='|', names=['title','year','a','b','c','location'])
#movie[["year"]] = movie[["year"]].apply(pd.to_numeric)
movielocations.location.value_counts().head(20)
#movie[movie.year < 1925].location.value_counts().head(20)

Los Angeles, California, USA                 20116
New York City, New York, USA                 10945
Mexico                                        7656
London, England, UK                           6428
Paris, France                                 4459
Toronto, Ontario, Canada                      4283
USA                                           3866
Berlin, Germany                               3625
Vancouver, British Columbia, Canada           3068
Chicago, Illinois, USA                        2956
San Francisco, California, USA                2859
Buenos Aires, Federal District, Argentina     2805
Madrid, Spain                                 2776
Montréal, Québec, Canada                      2254
Hollywood, Los Angeles, California, USA       2234
Rome, Lazio, Italy                            2093
New York, USA                                 2028
California, USA                               1837
Las Vegas, Nevada, USA                        1757
Barcelona, Catalonia, Spain    

In [107]:
geolocator = Nominatim(user_agent="moviedatabase")
location = geolocator.geocode("Toronto, Ontario, Canada")
location = geolocator.reverse(str(location.latitude)+","+str(location.longitude))
address = location.raw['address']
print(address)
city = address.get('city', '')
state = address.get('state', '')
country = address.get('country', '')
print('Address :', location)
print('City : ', city)
print('State : ', state)
print('Country : ', country)

{'amenity': 'Toronto City Hall', 'house_number': '100', 'road': 'Queen Street West', 'neighbourhood': 'Financial District', 'quarter': 'Spadina—Fort York', 'city': 'Old Toronto', 'state_district': 'Golden Horseshoe', 'state': 'Ontario', 'postcode': 'M5H 2N2', 'country': 'Canada', 'country_code': 'ca'}
Address : Toronto City Hall, 100, Queen Street West, Financial District, Spadina—Fort York, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5H 2N2, Canada
City :  Old Toronto
State :  Ontario
Country :  Canada


In [ ]:
movielocations = movielocations.drop(columns=['a','b','c'])
movielocations['year'] = movielocations['year'].fillna(0)
movielocations['year'] = movielocations['year'].astype(int)
movies['year'] = movies['year'].apply(pd.to_numeric, errors='coerce').dropna().astype(int)
movielocations = pd.merge(movies, movielocations)
movielocations = movielocations.drop(columns=['runtime','genres','title','year'])

In [113]:
movielocations['address'] = movielocations['location']
movielocations['city'] = ''
movielocations['state'] = ''
movielocations['country'] = ''
movielocations

,movieID,location,address,city,state,country
0,tt0000574,"Eltham, Victoria, Australia","Eltham, Victoria, Australia",,,
1,tt0000574,"Greensborough, Victoria, Australia","Greensborough, Victoria, Australia",,,
2,tt0000574,"Heidelberg, Victoria, Australia","Heidelberg, Victoria, Australia",,,
3,tt0000574,"Melbourne, Victoria, Australia","Melbourne, Victoria, Australia",,,
4,tt0000574,"Mitcham, Victoria, Australia","Mitcham, Victoria, Australia",,,
...,...,...,...,...,...,...
262733,tt9884086,"Algarrobo, Chile","Algarrobo, Chile",,,
262734,tt9884086,"Santiago, Chile","Santiago, Chile",,,
262735,tt9884086,"Port Deposit, Maryland, USA","Port Deposit, Maryland, USA",,,
262736,tt9887274,"Sacramento, California, USA","Sacramento, California, USA",,,


In [249]:
for index, row in movielocations[114101:].iterrows():
    if index % 5000 == 0:
        print(index)
        finallocations = movielocations.rename(columns={"location": "locationListing", "address": "rawAddress"})
        finallocations.to_csv('shootinglocation.txt', sep='\t', index=False)
    location = geolocator.geocode(row['location'], language='en',timeout=None)
    row['address'] = location
    if hasattr(location,'latitude') and location.latitude is not None and hasattr(location,'longitude') and location.longitude is not None:
        location = geolocator.reverse((location.latitude, location.longitude), language='en',timeout=None)
        if hasattr(location,'raw') and location.raw is not None:
            address = location.raw['address']
            city = address.get('city', '')
            state = address.get('state', '')
            country = address.get('country', '')
            row['city'] = city
            row['state'] = state
            row['country'] = country

115000


KeyboardInterrupt: 

In [251]:
finallocations = movielocations.rename(columns={"location": "locationListing", "address": "rawAddress"})
finallocations.to_csv('shootinglocation.txt', sep='\t', index=False)

In [5]:
starring = pd.read_csv('title.principals.tsv/data.tsv', header=None, sep='\t')
headers = starring.iloc[0]
starring = pd.DataFrame(starring.values[1:], columns=headers)
starring

C:\Users\andyc\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N
...,...,...,...,...,...,...
43906464,tt9916880,5,nm0996406,director,principal director,\N
43906465,tt9916880,6,nm1482639,writer,\N,\N
43906466,tt9916880,7,nm2586970,writer,books,\N
43906467,tt9916880,8,nm1594058,producer,producer,\N


In [79]:
movielocations.head(50)

,movieID,location
0,tt0000574,"Eltham, Victoria, Australia"
1,tt0000574,"Greensborough, Victoria, Australia"
2,tt0000574,"Heidelberg, Victoria, Australia"
3,tt0000574,"Melbourne, Victoria, Australia"
4,tt0000574,"Mitcham, Victoria, Australia"
5,tt0000574,"Rosanna, Victoria, Australia"
6,tt0000615,"New South Wales, Australia"
7,tt0000869,"La Garriga, Barcelona, Catalonia, Spain"
8,tt0001240,"Kronberg Castle, Denmark"
9,tt0001240,"Helsingør Kloster, Denmark"


In [6]:
starring = starring[(starring.category == 'actor') | (starring.category == 'actress')]

In [7]:
starring = starring.rename(columns={'tconst': "movieID", 'nconst': 'personID'})

In [8]:
starring = pd.merge(starring, movies)
starring

,movieID,ordering,personID,category,job,characters,title,year,runtime,genres
0,tt0000502,1,nm0215752,actor,\N,\N,Bohemios,1905,100,\N
1,tt0000502,2,nm0252720,actor,\N,\N,Bohemios,1905,100,\N
2,tt0000574,1,nm0846887,actress,\N,"[""Kate Kelly""]",The Story of the Kelly Gang,1906,70,"Action,Adventure,Biography"
3,tt0000574,2,nm0846894,actor,\N,"[""School Master""]",The Story of the Kelly Gang,1906,70,"Action,Adventure,Biography"
4,tt0000574,3,nm3002376,actor,\N,"[""Steve Hart""]",The Story of the Kelly Gang,1906,70,"Action,Adventure,Biography"
...,...,...,...,...,...,...,...,...,...,...
1752727,tt9916706,4,nm3272130,actress,\N,\N,Dankyavar Danka,2013,\N,Comedy
1752728,tt9916730,1,nm6096005,actor,\N,\N,6 Gunn,2017,116,\N
1752729,tt9916730,2,nm0059461,actor,\N,\N,6 Gunn,2017,116,\N
1752730,tt9916730,3,nm4852679,actor,\N,\N,6 Gunn,2017,116,\N


In [9]:
person = pd.read_csv('name.basics.tsv/persondata.tsv', header=None, sep='\t')

In [10]:
headers = person.iloc[0]
person = pd.DataFrame(person.values[1:], columns=headers)
person = person.drop(columns=['deathYear','primaryProfession','knownForTitles'])
person = person.rename(columns={'nconst': 'personID', 'primaryName': 'name', 'birthYear': 'birthYear'})
person

,personID,name,birthYear
0,nm0000001,Fred Astaire,1899
1,nm0000002,Lauren Bacall,1924
2,nm0000003,Brigitte Bardot,1934
3,nm0000004,John Belushi,1949
4,nm0000005,Ingmar Bergman,1918
...,...,...,...
10825471,nm9993714,Romeo del Rosario,\N
10825472,nm9993716,Essias Loberg,\N
10825473,nm9993717,Harikrishnan Rajan,\N
10825474,nm9993718,Aayush Nair,\N


In [72]:
person['birthYear'] = person.birthYear.str.replace(r"\\N", "NULL")
person

,personID,name,birthYear
0,nm0000001,Fred Astaire,1899
1,nm0000002,Lauren Bacall,1924
2,nm0000003,Brigitte Bardot,1934
3,nm0000004,John Belushi,1949
4,nm0000005,Ingmar Bergman,1918
...,...,...,...
10825471,nm9993714,Romeo del Rosario,NULL
10825472,nm9993716,Essias Loberg,NULL
10825473,nm9993717,Harikrishnan Rajan,NULL
10825474,nm9993718,Aayush Nair,NULL


In [11]:
starring = pd.merge(starring, person)
starring

,movieID,ordering,personID,category,job,characters,title,year,runtime,genres,name,birthYear
0,tt0000502,1,nm0215752,actor,\N,\N,Bohemios,1905,100,\N,Antonio del Pozo,\N
1,tt0000502,2,nm0252720,actor,\N,\N,Bohemios,1905,100,\N,El Mochuelo,\N
2,tt0000574,1,nm0846887,actress,\N,"[""Kate Kelly""]",The Story of the Kelly Gang,1906,70,"Action,Adventure,Biography",Elizabeth Tait,1879
3,tt0000574,2,nm0846894,actor,\N,"[""School Master""]",The Story of the Kelly Gang,1906,70,"Action,Adventure,Biography",John Tait,1871
4,tt0000574,3,nm3002376,actor,\N,"[""Steve Hart""]",The Story of the Kelly Gang,1906,70,"Action,Adventure,Biography",Norman Campbell,\N
...,...,...,...,...,...,...,...,...,...,...,...,...
1751179,tt9916362,4,nm10678594,actress,\N,"[""Katalin""]",Coven,2020,90,"Adventure,Drama,History",Garazi Urkola,\N
1751180,tt9916428,2,nm9445072,actor,\N,"[""Mao Ze Dong""]",The Secret of China,2019,\N,"Adventure,History,War",Wang Peng Kai,\N
1751181,tt9916428,3,nm8594703,actor,\N,"[""Dr. Hatem""]",The Secret of China,2019,\N,"Adventure,History,War",Valery Gadreau,\N
1751182,tt9916538,3,nm10041459,actor,\N,"[""Vikash""]",Kuambil Lagi Hatiku,2019,123,Drama,Sahil Shah,\N


In [12]:
starring = starring.drop(columns=['ordering', 'category', 'job', 'title', 'year', 'runtime', 'genres', 'birthYear'])
starring

,movieID,personID,characters,name
0,tt0000502,nm0215752,\N,Antonio del Pozo
1,tt0000502,nm0252720,\N,El Mochuelo
2,tt0000574,nm0846887,"[""Kate Kelly""]",Elizabeth Tait
3,tt0000574,nm0846894,"[""School Master""]",John Tait
4,tt0000574,nm3002376,"[""Steve Hart""]",Norman Campbell
...,...,...,...,...
1751179,tt9916362,nm10678594,"[""Katalin""]",Garazi Urkola
1751180,tt9916428,nm9445072,"[""Mao Ze Dong""]",Wang Peng Kai
1751181,tt9916428,nm8594703,"[""Dr. Hatem""]",Valery Gadreau
1751182,tt9916538,nm10041459,"[""Vikash""]",Sahil Shah


In [22]:
starring[950:1000]

,movieID,personID,characters,name
950,tt0013911,nm0002503,"[""Bob""]",Harry Carey
951,tt0013953,nm0002503,"[""Blake""]",Harry Carey
952,tt0013983,nm0002503,"[""Bob""]",Harry Carey
953,tt0014282,nm0002503,"[""Neil Allison""]",Harry Carey
954,tt0014308,nm0002503,"[""'The Hawk'""]",Harry Carey
955,tt0014907,nm0002503,"[""Bill Jones""]",Harry Carey
956,tt0015069,nm0002503,"[""Phlip Morgan""]",Harry Carey
957,tt0015108,nm0002503,\N,Harry Carey
958,tt0015284,nm0002503,"[""Big Bill Benson""]",Harry Carey
959,tt0015416,nm0002503,"[""Tiger Thompson""]",Harry Carey


In [32]:
starring.characters.str.replace('\\["', '').str.replace('\\"]', '').str.replace('","', ',')[starring.movieID == 'tt0016350']

964       Joel Parsons,Silent Sanderson
84631                    Judith Bensonn
86492                      Mrs. Parsons
111151                      Jim Downing
113247                      Art Parsons
Name: characters, dtype: object

In [33]:
starring['characters'] = starring.characters.str.replace('\\["', '').str.replace('\\"]', '').str.replace('","', ',')

In [59]:
a = starring.set_index(['movieID', 'personID', 'name']).apply(lambda x: x.str.split(',').explode()).reset_index()   

In [70]:
a = a.rename(columns={'characters': 'role'})
a['role'] = a.role.str.replace(r"\\N", "NULL")
a

,movieID,personID,name,role
0,tt0000502,nm0215752,Antonio del Pozo,NULL
1,tt0000502,nm0252720,El Mochuelo,NULL
2,tt0000574,nm0846887,Elizabeth Tait,Kate Kelly
3,tt0000574,nm0846894,John Tait,School Master
4,tt0000574,nm3002376,Norman Campbell,Steve Hart
...,...,...,...,...
1798930,tt9916362,nm10678594,Garazi Urkola,Katalin
1798931,tt9916428,nm9445072,Wang Peng Kai,Mao Ze Dong
1798932,tt9916428,nm8594703,Valery Gadreau,Dr. Hatem
1798933,tt9916538,nm10041459,Sahil Shah,Vikash


In [73]:
person.to_csv('person.txt', sep='\t', index=False)

In [71]:
a.to_csv('starring.txt', sep='\t', index=False)